In [1]:
# Load Libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
import pybaseball as pyb
from pybaseball import statcast
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

In [2]:
# Import Data
df = pd.read_csv("/Users/genoc/Desktop/HyannisHarborHawks/Trackman/CapeTrackmanDatabase.csv",low_memory=False)

In [3]:
# Clean Trackman Data
dft = df[['Pitcher', 'PitcherTeam', 'Batter', 'BatterTeam', 'PitcherThrows', 'RelSpeed','SpinRate','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak', 'PitchCall', 'PlayResult', 'TaggedHitType', 'KorBB']]

#Rename the columns to be more descriptive
dft.rename(columns={'BatterTeam':'Batter Team', 'PitcherTeam':'Pitcher Team', 'PitcherThrows':'P Side', 'RelSpeed':'Velocity', 'SpinRate':'Spin Rate', 'RelHeight':'Release Height', 'RelSide':'Release Side', 'Extension':'Extension', 'InducedVertBreak':'Induced Vertical Break', 'HorzBreak':'Horizontal Break', 'PitchCall':'Pitch Call', 'PlayResult':'Play Result', 'TaggedHitType':'Hit Type'}, inplace=True)

#Add the Pitch Type and Pitch Result to the new dataframe
dft.insert(1, 'Pitch Type', df['TaggedPitchType'])
dft.insert(2, 'Pitch Result', df['PitchCall'])

#Create a new column for the difference between the Induced Vertical Break and the Horizontal Break (Absolute Value)
dft['Differential Break'] = (dft['Induced Vertical Break'] - dft['Horizontal Break']).abs()

#Add two new columns for Pitch Location
dft.insert(10, 'Pitch Height', df['PlateLocHeight'])
dft.insert(11, 'Pitch Side', df['PlateLocSide'])

#Edit Horizontal Break to be absolute value of horizontal break
dft['Horizontal Break'] = abs(dft['Horizontal Break'])

# Edit the pitcher names so the first name is first and the last name is last
dft['Pitcher'] = dft['Pitcher'].str.split(' ', expand=True)[1] + ' ' + dft['Pitcher'].str.split(' ', expand=True)[0]

# Edit the batter names so the first name is first and the last name is last
dft['Batter'] = dft['Batter'].str.split(' ', expand=True)[1] + ' ' + dft['Batter'].str.split(' ', expand=True)[0]

# Remove the comma from the last name
dft = dft.replace(',', '', regex=True)

C:\Users\genoc\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-3-5b78fb82dc15>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft['Differential Break'] = (dft['Induced Vertical Break'] - dft['Horizontal Break']).abs()
<ipython-input-3-5b78fb82dc15>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [4]:
def categorize_os(dft):
    # Replace 'slider' with 'sweeper' or 'gyro' based on horizontal break values
    dft.loc[(dft['Pitch Type'] == 'Slider') & (dft['Horizontal Break'] > 14), 'Pitch Type'] = 'Sweeper'
    dft.loc[dft['Pitch Type'] == 'Slider', 'Pitch Type'] = 'Gyro'
    
    
    return dft

# Call the function to categorize sliders in the dataframe
dft = categorize_os(dft)

In [5]:
dft2 = dft[['Batter', 'Batter Team', 'Pitch Call', 'Play Result', 'Hit Type', 'KorBB']]

dft2['is_K'] = dft2['KorBB'].apply(lambda x: 1 if x == "Strikeout" else 0)
dft2['is_uBB'] = dft2.apply(lambda row: 1 if row['KorBB'] == "Walk" and (row['Pitch Call'] == "BallCalled" or row['Pitch Call'] == "BallinDirt") else 0, axis=1)
dft2['is_HBP'] = dft2['Pitch Call'].apply(lambda x: 1 if x == "HitByPitch" else 0)
dft2['is_1B'] = dft2['Play Result'].apply(lambda x: 1 if x == "Single" else 0)
dft2['is_2B'] = dft2['Play Result'].apply(lambda x: 1 if x == "Double" else 0)
dft2['is_3B'] = dft2['Play Result'].apply(lambda x: 1 if x == "Triple" else 0)
dft2['is_HR'] = dft2['Play Result'].apply(lambda x: 1 if x == "HomeRun" else 0)
dft2['is_iBB'] = dft2.apply(lambda row: 1 if row['KorBB'] == "Walk" and row['Pitch Call'] == "BallIntentional" else 0, axis=1)
dft2['is_SF'] = dft2.apply(lambda row: 1 if row['Hit Type'] == "FlyBall" and row['Play Result'] == "Sacrifice" else 0, axis=1)
dft2['is_ab'] = dft2.apply(lambda row: 1 if row['Play Result'] in ["Single", "Double", "Triple", "HomeRun", "Strikeout", "Error", "FieldersChoice", "Out"] or row['KorBB'] in ["Strikeout", "Walk"] else 0, axis=1)

# Group the data by player
grouped_data = dft2.groupby('Batter')

<ipython-input-5-d422e2649619>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft2['is_K'] = dft2['KorBB'].apply(lambda x: 1 if x == "Strikeout" else 0)
<ipython-input-5-d422e2649619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft2['is_uBB'] = dft2.apply(lambda row: 1 if row['KorBB'] == "Walk" and (row['Pitch Call'] == "BallCalled" or row['Pitch Call'] == "BallinDirt") else 0, axis=1)
<ipython-input-5-d422e2649619>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [6]:
# Calculate OPS+ for each player in the league and put it in a dictionary
player_ops_plus = {}
player_obp_plus = {}
player_slug_plus = {}

for player, group in grouped_data:
    at_bats_league = group['is_ab'].sum()
    hits_league = group['is_1B'].sum() + group['is_2B'].sum() + group['is_3B'].sum() + group['is_HR'].sum()
    walks_league = group['is_uBB'].sum() + group['is_iBB'].sum()
    total_bases_league = group['is_1B'].sum() + 2 * group['is_2B'].sum() + 3 * group['is_3B'].sum() + 4 * group['is_HR'].sum()

    on_base_percentage_league = (hits_league + walks_league) / at_bats_league if at_bats_league != 0 else 0
    slugging_percentage_league = total_bases_league / at_bats_league if at_bats_league != 0 else 0
    ops_league = on_base_percentage_league + slugging_percentage_league
    
    player_obp_plus[player] = round(on_base_percentage_league, 3)
    player_slug_plus[player] = round(slugging_percentage_league, 3)
    player_ops_plus[player] = round(ops_league, 3)

In [7]:
# Create a new DataFrame from player_ops_plus dictionary
df_ops = pd.DataFrame.from_dict(player_ops_plus, orient='index', columns=['OPS'])
df_obp = pd.DataFrame.from_dict(player_obp_plus, orient='index', columns=['OBP'])
df_slug = pd.DataFrame.from_dict(player_slug_plus, orient='index', columns=['SLUG'])


# Reset index to convert player names from index to a column
df_ops.reset_index(inplace=True)
df_ops.rename(columns={'index': 'Player'}, inplace=True)

df_slug.reset_index(inplace=True)
df_slug.rename(columns={'index': 'Player'}, inplace=True)

df_obp.reset_index(inplace=True)
df_obp.rename(columns={'index': 'Player'}, inplace=True)

# Calculate the average of all player_ops_plus values
league_average_ops = np.mean(list(player_ops_plus.values()))
league_average_obp = np.mean(list(player_obp_plus.values()))
league_average_slug = np.mean(list(player_slug_plus.values()))

In [8]:
dft1 = dft[['Batter', 'Batter Team', 'Pitch Call', 'Play Result', 'Hit Type', 'KorBB']]

dft_hya = dft1[(dft1['Batter Team'] == 'Hyannis')]

In [9]:
included_players = ['Brody Donay', 'Zach Ehrhard', 'Jonathan Gazdar', 'Elijah Hainline', 'Alex Lane', 'Trey Lipsey', 'Nick Mitchell', 'Bennett Thompson', 'Cameron Smith', 'Eric Snow', 'Will Taylor', 'Zachary Yorke']  # List of players to include

dft_hya = dft_hya[dft_hya['Batter'].isin(included_players)]

In [10]:
dft_hya['is_K'] = dft_hya['KorBB'].apply(lambda x: 1 if x == "Strikeout" else 0)
dft_hya['is_uBB'] = dft_hya.apply(lambda row: 1 if row['KorBB'] == "Walk" and (row['Pitch Call'] == "BallCalled" or row['Pitch Call'] == "BallinDirt") else 0, axis=1)
dft_hya['is_HBP'] = dft_hya['Pitch Call'].apply(lambda x: 1 if x == "HitByPitch" else 0)
dft_hya['is_1B'] = dft_hya['Play Result'].apply(lambda x: 1 if x == "Single" else 0)
dft_hya['is_2B'] = dft_hya['Play Result'].apply(lambda x: 1 if x == "Double" else 0)
dft_hya['is_3B'] = dft_hya['Play Result'].apply(lambda x: 1 if x == "Triple" else 0)
dft_hya['is_HR'] = dft_hya['Play Result'].apply(lambda x: 1 if x == "HomeRun" else 0)
dft_hya['is_iBB'] = dft_hya.apply(lambda row: 1 if row['KorBB'] == "Walk" and row['Pitch Call'] == "BallIntentional" else 0, axis=1)
dft_hya['is_SF'] = dft_hya.apply(lambda row: 1 if row['Hit Type'] == "FlyBall" and row['Play Result'] == "Sacrifice" else 0, axis=1)
dft_hya['is_ab'] = dft_hya.apply(lambda row: 1 if row['Play Result'] in ["Single", "Double", "Triple", "HomeRun", "Strikeout", "Error", "FieldersChoice", "Out"] or row['KorBB'] in ["Strikeout", "Walk"] else 0, axis=1)

# Group the data by player
grouped_hyannis_data = dft_hya.groupby('Batter')

In [11]:
# Calculate OPS for each player
hyannis_ops_plus = {}
hyannis_obp_plus = {}
hyannis_slug_plus = {}

for player, group in grouped_hyannis_data:
    at_bats = group['is_ab'].sum()
    hits = group['is_1B'].sum() + group['is_2B'].sum() + group['is_3B'].sum() + group['is_HR'].sum()
    walks = group['is_uBB'].sum() + group['is_iBB'].sum()
    total_bases = group['is_1B'].sum() + 2 * group['is_2B'].sum() + 3 * group['is_3B'].sum() + 4 * group['is_HR'].sum()

    on_base_percentage = (hits + walks) / at_bats if at_bats != 0 else 0
    slugging_percentage = total_bases / at_bats if at_bats != 0 else 0
    ops = on_base_percentage + slugging_percentage

    ops_plus = 100 * (ops/league_average_ops)
    slug_plus = 100 * (slugging_percentage/league_average_slug)
    obp_plus = 100 * (on_base_percentage/league_average_obp)
    
    hyannis_slug_plus[player] = slug_plus.round(1)
    hyannis_obp_plus[player] = obp_plus.round(1)
    hyannis_ops_plus[player] = ops_plus.round(1)
    
    

In [12]:
hyannis_obp_plus

{'Alex Lane': 121.0,
 'Bennett Thompson': 135.0,
 'Brody Donay': 98.4,
 'Cameron Smith': 118.4,
 'Elijah Hainline': 95.9,
 'Eric Snow': 85.8,
 'Jonathan Gazdar': 124.5,
 'Nick Mitchell': 107.2,
 'Trey Lipsey': 100.3,
 'Will Taylor': 97.8,
 'Zach Ehrhard': 125.6,
 'Zachary Yorke': 61.4}

In [13]:
hyannis_slug_plus

{'Alex Lane': 106.2,
 'Bennett Thompson': 82.2,
 'Brody Donay': 142.2,
 'Cameron Smith': 192.3,
 'Elijah Hainline': 82.1,
 'Eric Snow': 84.7,
 'Jonathan Gazdar': 144.0,
 'Nick Mitchell': 139.7,
 'Trey Lipsey': 81.4,
 'Will Taylor': 109.1,
 'Zach Ehrhard': 113.3,
 'Zachary Yorke': 68.5}

In [14]:
hyannis_ops_plus

{'Alex Lane': 114.0,
 'Bennett Thompson': 110.1,
 'Brody Donay': 119.1,
 'Cameron Smith': 153.3,
 'Elijah Hainline': 89.4,
 'Eric Snow': 85.3,
 'Jonathan Gazdar': 133.7,
 'Nick Mitchell': 122.6,
 'Trey Lipsey': 91.3,
 'Will Taylor': 103.1,
 'Zach Ehrhard': 119.8,
 'Zachary Yorke': 64.7}

In [20]:
df_ops = df_ops.round(3)
df_obp = df_obp.round(3)
df_slug = df_slug.round(3)

In [21]:
df_ops['OPS'].describe()

count    438.000000
mean       0.617829
std        0.412861
min        0.000000
25%        0.406000
50%        0.600000
75%        0.783750
max        3.000000
Name: OPS, dtype: float64

In [22]:
df_obp['OBP'].describe()

count    438.00000
mean       0.32589
std        0.19061
min        0.00000
25%        0.24450
50%        0.33300
75%        0.40000
max        1.00000
Name: OBP, dtype: float64

In [23]:
df_slug['SLUG'].describe()

count    438.000000
mean       0.291897
std        0.259580
min        0.000000
25%        0.149500
50%        0.251000
75%        0.392000
max        2.000000
Name: SLUG, dtype: float64